In [54]:
# %%
# 单元格 1: 安装/升级必要的库
# 运行此单元格将确保 pymatgen, mp-api (MPRester 的新后端) 和 pandas 都是最新版本
# 这将解决 NumPy 2.0 的兼容性问题
print("正在安装或更新 pymatgen, mp-api 和 pandas...")
!pip install --upgrade pymatgen mp-api pandas
print("安装完成！")

正在安装或更新 pymatgen, mp-api 和 pandas...
安装完成！


In [60]:
# %%
# 单元格 2: 定义你的 API 密钥 (Key)
# !!! 这是最重要的一步 !!!
# 
# !!! 请访问 [https://materials.project.org/dashboard](https://materials.project.org/dashboard) 重新生成一个新的 API Key !!!
#
# 2. 将 "YOUR_API_KEY_HERE" 替换为你自己的【新】 Key (保留引号)

# 【【【 请在这里替换成你自己的【全新】 API 密钥 】】】
MY_API_KEY = "G5eH6u1l8O6Zf6IvsEUiqcLqMoRIilhv"

if MY_API_KEY == "YOUR_API_KEY_HERE":
    print("!!! 警告：请先将 'YOUR_API_KEY_HERE' 替换为你从 MP 官网【新生成】的真实 API 密钥！")
else:
    print(f"API 密钥（{MY_API_KEY[:4]}...{MY_API_KEY[-4:]}）已准备就绪。")


API 密钥（G5eH...ilhv）已准备就绪。


In [64]:
# %%
# 单元格 3: 导入 MPRester 并初始化
# 【【【 核心修复 1：放弃 !pmg config，直接传递 API Key 】】】
# 导入Pymatgen中用于连接 MP 数据库的 "管理员"
from pymatgen.ext.matproj import MPRester

# 初始化 MPRester。
# 我们将单元格 2 中定义的 MY_API_KEY 直接作为参数传入
try:
    if MY_API_KEY == "YOUR_API_KEY_HERE":
        print("!!! 错误：请先在 单元格 2 中填入你【新】的 API 密钥再运行此单元格！")
        mpr = None # 显式设置 mpr 为 None
    else:
        mpr = MPRester(MY_API_KEY)
        # 【【【 核心修复 5：移除 'fields' 参数，API 不支持在简单查询中指定 'fields' 】】】
        # 我们只查询 'Fe2O3'，不指定字段，来测试连接
        print("正在测试连接...")
        test_search = mpr.summary.search(formula="Fe2O3")
        if test_search:
            # 【【【 核心修复 8：.material_id 改为 ['material_id'] 】】】
            # 不指定 fields 时, 返回的是 dict 列表, 需用 [] 访问
            print(f"MPRester 初始化成功！已连接到 MP 数据库。(测试连接成功，Fe2O3 的一个 ID 是 {test_search[0]['material_id']})")
        else:
             print("MPRester 初始化成功！但测试查询 Fe2O3 未返回结果。")
except Exception as e:
    print(f"MPRester 初始化失败，请检查：\n1. API 密钥是否正确且为【新】密钥？\n2. 网络连接是否正常？\n错误信息: {e}")
    mpr = None # 初始化失败


正在测试连接...
MPRester 初始化成功！已连接到 MP 数据库。(测试连接成功，Fe2O3 的一个 ID 是 mp-1244869)


In [66]:
# %%
# 单元格 4: 牛刀小试 - 按化学式查询 (Query by Formula)
# 【【【 核心修复 9：移除 'fields' 参数以避免 API 错误 】】】
# 让我们来查一下 "Fe2O3" (氧化铁)
formula = "Fe2O3"
print(f"正在查询化学式为 {formula} 的材料...")

try:
    if mpr is None:
         print("!!! 错误：MPRester 未成功初始化，请检查 单元格 3。")
         fe2o3_data = [] # 定义一个空列表以避免 NameError
    else:
        # 【【【 注意：移除 fields=[...]，因为 API 不允许 formula 和 fields C同时使用 】】】
        fe2o3_data_docs = mpr.summary.search(
            formula=formula
            # fields=["material_id", "formula_pretty", "energy_per_atom", "band_gap", "is_stable"] # <-- 此行被移除
        )
        
        # 【【【 修复：返回的是 dict 列表，不再需要 .model_dump() 】】】
        fe2o3_data = fe2o3_data_docs # 直接赋值

        # 【【【 核心修复 6：修正笔误 len(fe2T_DATA_DOCSo3_data) 】】】
        print(f"查询完成！共找到 {len(fe2o3_data)} 条 {formula} 的记录（代表不同晶相）。")

        # 让我们打印前 5 条看看
        # 【【【 修复：使用字典方式 ['key'] 访问数据 】】】
        print("\n--- Fe2O3 查询结果 (前 5 条) ---")
        for entry in fe2o3_data[:5]:
            # .get('key', 'N/A') 是一种更安全的方式，如果某个键不存在，它会返回 'N/A' 而不是报错
            print(f"ID: {entry.get('material_id', 'N/A')}, "
                  f"Formula: {entry.get('formula_pretty', 'N/A')}, "
                  f"Stable: {entry.get('is_stable', 'N/A')}, "
                  f"Band Gap: {entry.get('band_gap', 'N/A')}")
except Exception as e:
    print(f"查询失败。错误: {e}")
    fe2o3_data = [] # 确保在出错时 fe2o3_data 仍然被定义


正在查询化学式为 Fe2O3 的材料...
查询完成！共找到 26 条 Fe2O3 的记录（代表不同晶相）。

--- Fe2O3 查询结果 (前 5 条) ---
ID: mp-1244869, Formula: Fe2O3, Stable: False, Band Gap: 0.220199999999999
ID: mp-1244911, Formula: Fe2O3, Stable: False, Band Gap: 0.283199999999999
ID: mp-1245019, Formula: Fe2O3, Stable: False, Band Gap: 1.1378
ID: mp-1245078, Formula: Fe2O3, Stable: False, Band Gap: 1.1123
ID: mp-1245084, Formula: Fe2O3, Stable: False, Band Gap: 0.12109999999999901


In [67]:
# %%
# 单元格 5: 将查询结果转换为 Pandas DataFrame
# (此单元格的 'NameError' 将被自动修复)
# Python 列表不太直观，我们用 Pandas (Python中的"Excel") 把它变成表格
import pandas as pd

if not fe2o3_data:
    print("警告：'fe2O3_data' 为空，无法创建 DataFrame。请检查 单元格 4 的查询结果。")
    df_fe2o3 = pd.DataFrame() # 创建一个空的 DataFrame
else:
    # 一键转换为 DataFrame
    df_fe2o3 = pd.DataFrame(fe2o3_data)

    # 打印表格的头部
    print("\n--- Fe2O3 查询结果 (表格化) ---")
    # 打印时只显示我们关心的几列
    cols_to_show = ['material_id', 'formula_pretty', 'is_stable', 'band_gap', 'energy_per_atom']
    # 筛选出数据中实际存在的列，避免KeyError
    existing_cols = [col for col in cols_to_show if col in df_fe2o3.columns]
    print(df_fe2o3[existing_cols].head())



--- Fe2O3 查询结果 (表格化) ---
  material_id formula_pretty  is_stable  band_gap  energy_per_atom
0  mp-1244869          Fe2O3      False    0.2202        -7.774367
1  mp-1244911          Fe2O3      False    0.2832        -7.789982
2  mp-1245019          Fe2O3      False    1.1378        -7.775224
3  mp-1245078          Fe2O3      False    1.1123        -7.758055
4  mp-1245084          Fe2O3      False    0.1211        -7.511759


In [69]:
# %%
# 单元格 6: 实战核心 - 批量查询与筛选
# 【【【 策略调整：移除 is_stable=True，保证获取数据 】】】
# 我们将下载所有材料，稍后在 单元格 7 中用 Pandas 手动筛选
print("\n--- 开始批量查询与筛选任务 ---")

# 1. 定义我们的目标体系 (化学体系, chemsys)
target_systems = ['Li-Fe-O', 'Na-Co-O', 'Mg-S']

# 2. 定义我们感兴趣的属性（*注意：这现在只作为“注释”提醒我们关心什么*）
target_fields = [
    "material_id",
    "formula_pretty",
    "band_gap",
    "density",
    "volume",
    "is_stable",
    "e_above_hull", # 离凸包的能量，0 表示稳定
    # "structure" # <-- 默认不返回 structure
]

# 3. 创建一个空列表，用于存放所有结果
all_materials = []

# 4. 开始循环查询
if mpr is None:
    print("!!! 错误：MPRester 未成功初始化，跳过批量查询。请检查 单元格 3。")
else:
    for sys in target_systems:
        print(f"\n正在查询体系: {sys} ...")
        
        try:
            # 【【【 最终修复：移除 fields=target_fields 】】】
            # API 不允许 chemsys 和 fields 同时使用
            system_data_docs = mpr.summary.search(
                chemsys=sys
                # is_stable=True, # <-- 我们先把这个注释掉
                # fields=target_fields # <-- 【【【 此行被移除 】】】
            )
            
            # 【【【 最终修复：返回的是 dict 列表，不再需要 .model_dump() 】】】
            system_data = system_data_docs # 直接赋值
            
            print(f"体系 {sys} 查询到 {len(system_data)} 种材料（包含稳定和非稳定）。")
            
            # 将这个体系的结果添加到总列表中
            all_materials.extend(system_data)
        except Exception as e:
            print(f"查询体系 {sys} 失败。错误: {e}")


print(f"\n--- 批量查询全部完成！共找到 {len(all_materials)} 种材料 ---")



--- 开始批量查询与筛选任务 ---

正在查询体系: Li-Fe-O ...
体系 Li-Fe-O 查询到 176 种材料（包含稳定和非稳定）。

正在查询体系: Na-Co-O ...
体系 Na-Co-O 查询到 109 种材料（包含稳定和非稳定）。

正在查询体系: Mg-S ...
体系 Mg-S 查询到 7 种材料（包含稳定和非稳定）。

--- 批量查询全部完成！共找到 292 种材料 ---


In [72]:
# %%
# 单元格 7: 最终数据处理与分析
# 同样，我们将所有结果转换为 DataFrame
if not all_materials:
    print("警告：'all_materials' 为空，无法创建 DataFrame。请检查 单元格 6 的查询结果。")
    df_all = pd.DataFrame() # 创建一个空的 DataFrame
else:
    # 【【【 核心修复 7：修正笔误 all_modules 】】】
    df_all = pd.DataFrame(all_materials)
    print("\n--- 批量查询总表 (前10行) ---")
    print(df_all.head(10))

    # ---------------------------------------------------
    # 【【【 在本地用 Pandas 进行灵活筛选 】】】
    # ---------------------------------------------------

    # 筛选条件 1：只看热力学稳定相 (is_stable == True)
    stable_df = df_all[df_all['is_stable'] == True]
    print(f"\n--- 本地筛选：其中有 {len(stable_df)} 种是热力学稳定相 (is_stable=True) ---")
    print(stable_df.head())

    # 筛选条件 2：更灵活的“准稳定相” (能量在凸包 0.1 eV/atom 以内)
    # 这在科研中更常用！
    # 【【【 最终修复：修正拼写错误 'e_above_hull' -> 'energy_above_hull' 】】】
    quasi_stable_df = df_all[df_all['energy_above_hull'] <= 0.1]
    print(f"\n--- 本地筛选：其中有 {len(quasi_stable_df)} 种是准稳定相 (energy_above_hull <= 0.1 eV/atom) ---")
    print(quasi_stable_df.head())
    
    # 筛选条件 3：我们只关心上述“准稳定相”中的“半导体”（带隙 > 0.1 eV）
    target_semiconductors = quasi_stable_df[quasi_stable_df['band_gap'] > 0.1]
    print(f"\n--- 最终目标：{len(target_semiconductors)} 种 准稳定半导体材料 ---")
    print(target_semiconductors)



--- 批量查询总表 (前10行) ---
                                        builder_meta  nsites     elements  \
0  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      42  [Fe, Li, O]   
1  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      21  [Fe, Li, O]   
2  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      21  [Fe, Li, O]   
3  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      21  [Fe, Li, O]   
4  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      21  [Fe, Li, O]   
5  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      42  [Fe, Li, O]   
6  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      42  [Fe, Li, O]   
7  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...       7  [Fe, Li, O]   
8  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      28  [Fe, Li, O]   
9  {'emmet_version': '0.84.3rc4', 'pymatgen_versi...      14  [Fe, Li, O]   

   nelements                         composition  \
0          3  {'Li': 2.0, 'Fe': 16.0, 'O': 24.0}   
1          3   {'Li': 1.0

In [73]:
# %%
# 单元格 8: 保存成果到 CSV 文件
# 将我们筛选好的数据保存到本地，方便后续用 Origin, Excel 等软件分析

output_filename = "target_semiconductors_dataset.csv"

# 我们需要检查 'target_semiconductors' 是否在 单元格 7 中被成功定义
if 'target_semiconductors' in locals() and not target_semiconductors.empty:
    target_semiconductors.to_csv(output_filename, index=False) # index=False 表示不保存 DataFrame 的行索引
    print(f"\n--- 任务完成！---")
    print(f"所有 {len(target_semiconductors)} 条目标材料的数据已保存到文件: {output_filename}")
else:
    print(f"\n数据为空或 'target_semiconductors' 未定义，未保存文件 {output_filename}。")

# 你可以从 VS Code 左侧的文件浏览器中找到这个 .csv 文件



--- 任务完成！---
所有 127 条目标材料的数据已保存到文件: target_semiconductors_dataset.csv


In [74]:
# %%
# 单元格 4: 高级查询 - 定义“多重筛选”条件
print("--- 正在构建高级查询条件 ---")

# 1. 化学体系
chemsys = "Li-Co-O"

# 2. 元素种类数 (nelements)
# 我们要求 >= 3 (即 Li, Co, O 三元)
# MongoDB 语法：{"$gte": 3} (Greater Than or Equal)
nelements = {"$gte": 3}

# 3. 热力学稳定性 (energy_above_hull)
# 我们要求 <= 0.05 eV/atom
# MongoDB 语法：{"$lte": 0.05} (Less Than or Equal)
e_above_hull = {"$lte": 0.05}

# 4. 电学性质 (band_gap)
# 我们要求 > 0.1 eV (排除金属)
# MongoDB 语法：{"$gt": 0.1} (Greater Than)
band_gap = {"$gt": 0.1}

# 打印我们的查询条件，确认一下
print(f"查询 1 (体系): chemsys = {chemsys}")
print(f"查询 2 (元素数): nelements = {nelements}")
print(f"查询 3 (稳定性): energy_above_hull = {e_above_hull}")
print(f"查询 4 (带隙): band_gap = {band_gap}")
print("\n高级查询条件已准备就绪。")

--- 正在构建高级查询条件 ---
查询 1 (体系): chemsys = Li-Co-O
查询 2 (元素数): nelements = {'$gte': 3}
查询 3 (稳定性): energy_above_hull = {'$lte': 0.05}
查询 4 (带隙): band_gap = {'$gt': 0.1}

高级查询条件已准备就绪。


In [75]:
# %%
# 单元格 5: 执行“多条件”高级查询
print(f"--- 正在高通量筛选 {chemsys} 体系 ---")

try:
    if mpr is None:
         print("!!! 错误：MPRester 未成功初始化，请检查 单元格 3。")
         lco_data = [] # 定义一个空列表以避免 NameError
    else:
        # 将 单元格 4 的所有条件作为参数传入！
        lco_data_docs = mpr.summary.search(
            chemsys=chemsys,
            nelements=nelements,
            energy_above_hull=e_above_hull,
            band_gap=band_gap
            # 注意：我们没有使用 'fields'，所以将收到 dict 列表
        )
    
        # 直接赋值 (返回的是 dict 列表)
        lco_data = lco_data_docs 
    
        if not lco_data:
            print("\n查询成功，但未找到同时满足【所有】条件的材料。")
            print("这很正常，说明我们的筛选条件非常严格。")
        else:
            print(f"\n查询成功！共找到 {len(lco_data)} 种满足所有条件的材料！")
        
except Exception as e:
    print(f"查询失败。错误: {e}")
    lco_data = [] # 确保在出错时 lco_data 仍然被定义

--- 正在高通量筛选 Li-Co-O 体系 ---
查询失败。错误: REST query returned with error status code 422. Content: b'{"detail":[{"type":"int_parsing","loc":["query","nelements"],"msg":"Input should be a valid integer, unable to parse string as an integer","input":"$gte"}]}'
